In [21]:
import numpy as np
import pandas as pd

In [22]:
# open final data
data = pd.read_csv('../../data/processed/final_data.csv')

# get the list of columns
columns = data.columns
positive_columns = [col for col in columns if '+' in col]
negative_columns = [col for col in columns if '-' in col]
original_columns = [col for col in columns if '+' not in col and '-' not in col]

In [23]:
# use only the training data
model_training_data = data[data['dataset'] == 'train']

# convert the datetime columns to datetime
model_training_data['datetime'] = pd.to_datetime(model_training_data['datetime'])

/var/folders/bn/prwtql5x5sg0mbkxc6508fqm0000gn/T/ipykernel_83210/545332924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_training_data['datetime'] = pd.to_datetime(model_training_data['datetime'])


In [24]:
# mask the data from day 3 to 15 of each month using 'datetime' column
negative_train_mask = model_training_data['datetime'].apply(lambda x: x.day >= 3 and x.day <= 15)
negative_test_mask = model_training_data['datetime'].apply(lambda x: x.day > 15)

# get the negative training data
negative_train_data = model_training_data[negative_train_mask][original_columns + negative_columns].copy()
negative_test_data = model_training_data[negative_test_mask][original_columns + negative_columns].copy()

# maske the data from day 5 to 17 of each month using 'datetime' column
positive_train_mask = model_training_data['datetime'].apply(lambda x: x.day >= 5 and x.day <= 17)
positive_test_mask = model_training_data['datetime'].apply(lambda x: x.day < 5)

# get the positive training data
positive_train_data = model_training_data[positive_train_mask][original_columns + positive_columns].copy()
positive_test_data = model_training_data[positive_test_mask][original_columns + positive_columns].copy()

In [25]:
# for each train or test dataset show missing values per row
print('Negative training data')
print(negative_train_data.isnull().sum())
print('Negative test data')
print(negative_test_data.isnull().sum())
print('Positive training data')
print(positive_train_data.isnull().sum())
print('Positive test data')
print(positive_test_data.isnull().sum())

Negative training data
datetime          0
season            0
holiday           0
workingday        0
weather           0
temp              0
atemp             0
humidity          0
windspeed         0
casual            0
registered        0
count             0
year              0
month             0
day               0
hour              0
dataset           0
dayofyear         0
weekofyear        0
dayofweek         0
windspeed, -1     0
atemp, -1         0
humidity, -1      0
holiday, -1       0
workingday, -1    0
weather, -1       0
temp, -1          0
windspeed, -2     0
atemp, -2         0
humidity, -2      0
holiday, -2       0
workingday, -2    0
weather, -2       0
temp, -2          0
dtype: int64
Negative test data
datetime          0
season            0
holiday           0
workingday        0
weather           0
temp              0
atemp             0
humidity          0
windspeed         0
casual            0
registered        0
count             0
year              0
month